In [16]:
import nibabel as nib
from pathlib import Path
import numpy as np
import pandas as pd
import re

In [22]:
def create_atlas_dicitonary(path_to_atlas_desc_file):

    #atlas_file = Path("/home-local/kimm58/AtlasToDiffusionReg/data/JHU_MNI_SS_WMPM_Type-III_SlicerLUT.txt")
    atlas_doc = np.loadtxt(path_to_atlas_desc_file, dtype=str, delimiter='\n')
    atlas_dict = dict()
    for label in atlas_doc[1:]:   #assuming that the labelmap desciption must have a header
        splits = label.split(" ")
        atlas_dict[int(splits[0])] = splits[1]
    return atlas_dict

#get list of atlas names from the inputs folder
def get_atlas_names(path):
    atlases = []
    for f in [x.name for x in path.glob('*')]:
        name_match = re.search("^(Atlas.*).nii.*$", f, re.IGNORECASE)
        if name_match:
            atlases.append(name_match.group(1))
    return atlases

# for x in atlas_dict:
#     print(x, atlas_dict[x])

In [24]:
#file_name = [x for x in Path("/INPUTS").glob('*.bval*')][0].name
inp = Path("/home-local/kimm58/AtlasToDiffusionReg/data/inputs")
out = Path("/home-local/kimm58/AtlasToDiffusionReg/data/outputs")
file_name = [x for x in inp.glob('*.bval*')][0].name
name_match = re.search("^(.*).bval$", file_name)
name = name_match.group(1)

#inputs/intermediates
fa = Path("/OUTPUTS/{}%fa.nii.gz".format(name))
md = Path("/OUTPUTS/{}%md.nii.gz".format(name))
ad = Path("/OUTPUTS/{}%ad.nii.gz".format(name))
rd = Path("/OUTPUTS/{}%rd.nii.gz".format(name))


#creating the atlas labels dictionary --> will need to change the input names for this, perhaps 'Labels_...' instead of 'Atlas_...' for these .txt files
atlas_I_file = Path("/home-local/kimm58/AtlasToDiffusionReg/data/JHU_MNI_SS_WMPM_Type-III_SlicerLUT.txt")
atlas_I_dict = create_atlas_dicitonary(atlas_I_file)
#atlas_dicts = [atlas_I_dict, ...]

# for key,val in atlas_I_dict.items():
#     print(key, val)

metric_dict = dict()

scalar_files = [fa, md, ad, rd]
scalar_prefixes = ['fa','md','ad','rd']

#get atlas names
atlases = get_atlas_names(inp)

#calculate the scalars for each ROI
for scalar_name,scalar_prefix in zip(scalar_files, scalar_prefixes): #loop through all the scalar niftis (FA, MD, etc.)

    #scalar_file = Path("/OUTPUTS/{}%{}.nii.gz".format(name,scalar_prefix))  #the scalar nifti
    scalar_file = out/("{}%{}.nii.gz".format(name,scalar_prefix))
    assert scalar_file.exists(), "ERROR: scalar file {} does not exist".format(scalar_file)
    #scalar_nifti = nib.load(scalar_file)
    
    #will eventually need to loop through the atlases in the input/outputs dir
    #for atlas_dict, label_nii





AssertionError: ERROR: scalar file /OUTPUTS/dwmri%fa.nii.gz does not exist